In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

from overcooked_ai.dataset_types import DetectionDataset

HOME_DIR = Path(os.environ.get("HOME", "/home/mimic"))
SOURCE_DIR: Path = HOME_DIR / "Overcooked2_1-1_jpeg/"
ds_gt_path = SOURCE_DIR / "detection_dataset.mar2025.json"
ds_gt = DetectionDataset.load_from_json(ds_gt_path)

In [94]:
# Estimate camera intrinsics and distortion from 2D-3D correspondences using OpenCV

import numpy as np
import cv2

from overcooked_ai.game_maps import world_1_1_tile_object_ids
from overcooked_ai.grid_homography import apply_homography, match_euclidean_dist_linear_sum_assignment_xycats
from overcooked_ai.dataset_ops import convert_from_annotations_to_frame_coord_xycats, filter_tile_annotations
from overcooked_ai.type_conversions import convert_from_world_tiles_to_xycats

world_1_1_grid_xycats = convert_from_world_tiles_to_xycats(world_1_1_tile_object_ids)

per_entry_frame_space_xys: list[list[tuple[float, float]]] = []
per_entry_grid_space_xyzs: list[list[tuple[float, float, float]]] = []

for entry in ds_gt.entries[::5]:
    # Load homography
    H_grid_frame = np.array(entry.H_grid_img_vector).reshape(3, 3)
    H_frame_grid = np.linalg.inv(H_grid_frame)

    # Convert tile annotations into grid coordinates
    tile_frame_xycats = convert_from_annotations_to_frame_coord_xycats(filter_tile_annotations(entry.annotations))
    tile_frame_hxys = np.hstack((tile_frame_xycats[:, :2], np.ones((tile_frame_xycats.shape[0], 1))))
    tile_grid_hxys = apply_homography(H_frame_grid, tile_frame_hxys)
    tile_grid_xycats = np.hstack((tile_grid_hxys[:, :2], tile_frame_xycats[:, 2:]))

    # Associate with ground truth tile labels, and construct 2D-3D correspondences
    matched_tile_to_gt_idxs = match_euclidean_dist_linear_sum_assignment_xycats(tile_grid_xycats, world_1_1_grid_xycats)
    frame_space_xys = []
    grid_space_xyzs = []
    for tile_idx, gt_idx in matched_tile_to_gt_idxs:
        frame_space_xys.append((tile_frame_xycats[tile_idx, 0], tile_frame_xycats[tile_idx, 1]))
        grid_space_xyzs.append((world_1_1_grid_xycats[gt_idx, 0], world_1_1_grid_xycats[gt_idx, 1], 0))
    per_entry_frame_space_xys.append(np.array(frame_space_xys, dtype=np.float32))
    per_entry_grid_space_xyzs.append(np.array(grid_space_xyzs, dtype=np.float32))

In [95]:
flags = (
    cv2.CALIB_ZERO_TANGENT_DIST |
    cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K3 |
    cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5 | cv2.CALIB_FIX_K6
)
intrinsics_matrix_guess = None
distortion_coefficients_guess = np.zeros((1, 5))
rms_reprojection_error, K, distortion_coefficients, rotation_vectors, translation_vectors = cv2.calibrateCamera(
    objectPoints=per_entry_grid_space_xyzs,
    imagePoints=per_entry_frame_space_xys,
    imageSize=(entry.width, entry.height),
    cameraMatrix=intrinsics_matrix_guess,
    distCoeffs=distortion_coefficients_guess,
    flags=flags,
)
print("RMS reprojection error:\n", rms_reprojection_error)
print("Intrinsics matrix:\n", K)
print("Distortion coefficients:\n", distortion_coefficients)

RMS reprojection error:
 149.46383885031977
Intrinsics matrix:
 [[221.14738991   0.         909.61721633]
 [  0.         218.13983865 961.03971585]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[0. 0. 0. 0. 0.]]


In [114]:
# TODO: K above is stable when I use fewer/more frames. but how do I know if K looks good?

def decompose_homography(H, K):
    B = np.linalg.inv(K) @ H

    # Normalize scale using the first column
    norm = np.sqrt(np.linalg.norm(B[:, 0]) * np.linalg.norm(B[:, 1]))
    r1 = B[:, 0] / norm
    r2 = B[:, 1] / norm
    t  = B[:, 2] / norm
    r3 = np.cross(r1, r2)

    # Project R into SO(3)
    R_approx = np.stack([r1, r2, r3], axis=1)
    U, _, Vt = np.linalg.svd(R_approx)
    R = U @ Vt

    if np.linalg.det(R) < 0:
        R *= -1  # fix improper rotation

    return R, t

def project_points(grid_space_xyzs, K, R, t):
    N = grid_space_xyzs.shape[0]
    X_hom = np.hstack([grid_space_xyzs, np.ones((N, 1))])  # shape: (N, 4)
    P = K @ np.hstack([R, t.reshape(3, 1)])  # 3x4 projection matrix

    x_proj = (P @ X_hom.T).T  # (N, 3)
    x_proj /= x_proj[:, [2]]  # normalize
    return x_proj[:, :2]

R, t = decompose_homography(H_frame_grid, K)
X_proj = project_points(per_entry_grid_space_xyzs[0], K, R, t)
gt_xys = per_entry_frame_space_xys[0]
err = np.linalg.norm(X_proj - gt_xys, axis=1)
err.mean()


# def reprojection_error_K_eval(K, H, X_grids, x_images):
#     total_err = 0
#     total_pts = 0

#     for X_grid, x_img in zip(X_grids, x_images):
#         R, t = decompose_homography(H, K)
#         x_proj = project_points(X_grid, K, R, t)
#         err = np.linalg.norm(x_proj - x_img, axis=1)
#         total_err += np.sum(err)
#         total_pts += len(err)

#     return total_err / total_pts  # mean reprojection error in pixels

np.float64(419.3652582445317)

array([[ 398.5    ,  215.5    ],
       [ 482.5    ,  214.5    ],
       [ 571.5    ,  216.5    ],
       [ 660.     ,  214.5    ],
       [ 750.5    ,  212.5    ],
       [1099.     ,  210.5    ],
       [1188.5    ,  215.     ],
       [1276.     ,  215.5    ],
       [1365.     ,  216.     ],
       [1448.5    ,  215.     ],
       [ 381.5    ,  278.     ],
       [1459.5    ,  274.     ],
       [ 339.5    ,  425.5    ],
       [ 289.     ,  600.     ],
       [ 265.     ,  698.     ],
       [1567.5    ,  697.5    ],
       [ 232.5    ,  802.     ],
       [1605.     ,  802.     ],
       [1483.3646 ,  344.7259 ],
       [1502.8273 ,  422.90445],
       [ 363.     ,  351.     ],
       [ 337.     ,  802.     ],
       [ 460.5    ,  808.     ],
       [1370.     ,  802.     ],
       [1484.     ,  802.     ],
       [1524.5    ,  511.5    ],
       [1542.5    ,  599.     ],
       [ 318.5    ,  509.5    ]], dtype=float32)

In [ ]:
# Estimate camera intrinsics using Zhang's method
# TODO: why is this implementation wonky?

import scipy.linalg

def extract_v_ij(H: np.ndarray, i: int, j: int) -> np.ndarray:
    v_ij = np.array([
        H[0, i] * H[0, j],
        H[0, i] * H[1, j] + H[1, i] * H[0, j],
        H[1, i] * H[1, j],
        H[2, i] * H[0, j] + H[0, i] * H[2, j],
        H[2, i] * H[1, j] + H[1, i] * H[2, j],
        H[2, i] * H[2, j]
    ])
    return v_ij

B_entries = []
for entry in ds_gt.entries:
    # Load homography
    H_grid_frame = np.array(entry.H_grid_img_vector).reshape(3, 3)
    H_frame_grid = np.linalg.inv(H_grid_frame)
    H_frame_grid /= H_frame_grid[2, 2]

    B_entries.append(extract_v_ij(H_frame_grid, 0, 1))
    B_entries.append(extract_v_ij(H_frame_grid, 0, 0) - extract_v_ij(H_frame_grid, 1, 1))

B_mat = np.array(B_entries)
_, _, V = scipy.linalg.svd(B_mat)
b_vec = V[-1, :]
B = np.array(
    [[b_vec[0], b_vec[1], b_vec[3]],
     [b_vec[1], b_vec[2], b_vec[4]],
     [b_vec[3], b_vec[4], b_vec[5]]])

def nearest_positive_definite(A):
    """Find the nearest positive-definite matrix to A."""
    B = (A + A.T) / 2
    eigvals, eigvecs = np.linalg.eigh(B)
    eigvals_clipped = np.clip(eigvals, 1e-8, None)  # force positive
    return eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T

def compute_K_from_B(B):
    B11, B12, B13 = B[0, 0], B[0, 1], B[0, 2]
    B22, B23 = B[1, 1], B[1, 2]
    B33 = B[2, 2]

    v0 = (B12 * B13 - B11 * B23) / (B11 * B22 - B12**2)
    λ = B33 - (B13**2 + v0 * (B12 * B13 - B11 * B23)) / B11
    α = np.sqrt(λ / B11)
    β = np.sqrt(λ * B11 / (B11 * B22 - B12**2))
    γ = -B12 * α**2 * β / λ
    u0 = γ * v0 / β - B13 * α**2 / λ

    K = np.array([
        [α, γ, u0],
        [0, β, v0],
        [0, 0, 1]
    ])
    return K

B = nearest_positive_definite(B)

K = compute_K_from_B(B)
print(K)


[[ 1.72285969e+03  1.18724790e+02  2.08699011e+01]
 [ 0.00000000e+00  2.99006863e+02 -1.70099106e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
